### Figure 5 | Vaccination elicits coordinated pathway-level immune and metabolic responses in sarcoidosis
Pathway enrichment analyses summarize coordinated immune, metabolic, and stress-related transcriptional responses following vaccination.